In [ ]:
import time
import joblib
import nbimporter
from tensorflow.keras.models import load_model

from feature_extraction import compute_bandpower, compute_stat_features

In [ ]:
sf = 128 # Hz
window_size = 10 # seconds
model = load_model('../models/lstm.keras')
scalar = joblib.load('../models/scalar_2d.pkl')

In [ ]:
matrix = []
start_time = time.time()

while time.time() - start_time <= 60:
    row = []
    window = []
    window_start = time.time()

    while time.time() - window_start <= window_size:
        time.sleep(1/128)
        sample, _ = inlet.pull_sample()
        window.append(sample)

    window = np.array(window)

    for col in range(0, window.shape[1]):
        channel = window[:, col]
        theta = compute_bandpower(channel, 'theta')
        alpha = compute_bandpower(channel, 'alpha')
        beta = compute_bandpower(channel, 'beta')
        gamma = compute_bandpower(channel, 'gamma')

        beta_alpha_ratio = beta/alpha if alpha!=0 else 0

        mean, std, skewness, kurtosis, entropy = compute_stat_features(channel)

        row.extend([theta, alpha, beta, gamma, beta_alpha_ratio, mean, std, skewness, kurtosis, entropy])

    matrix.append(row)

matrix = scalar.transform(matrix)
matrix = np.array(matrix).reshape(1, 6, 320)

prediction = model.predict(matrix)
focus = int(np.round(prediction))

if focus == 1:
    output = 2

else:
    output = 0